#KAFKA PRODUCER PARA SUPLEMENTOS DEPORTIVOS "**GAINSIGHT**"

###Este notebook extrae datos de suplementos deportivos usando SerpAPI (Google Shopping) y los envía a un tópico de Kafka para procesamiento en tiempo real.

###Arquitectura: SerpAPI → Python → Kafka → MongoDB → Streamlit Dashboard


# Importación de Librerías

 Importamos todas las librerías necesarias para:
 -  **Kafka**: Productor de mensajes
 -  **Requests**: Llamadas a SerpAPI
 -  **JSON**: Serialización de datos
 -  **Time/Datetime**: Control de tiempo y timestamps

In [ ]:
from kafka import KafkaProducer
import json
import requests
from time import sleep
from datetime import datetime

# Configuración de SerpAPI

SerpAPI nos permite acceder a los resultados de Google Shopping de forma estructurada.
 ### Parámetros importantes:
-  **engine**: "google_shopping" para búsquedas de productos
-  **hl**: "es" para idioma español
-  **gl**: "us" para mejores resultados en suplementos (mercado más amplio)
-  **format**: "json" para respuesta estructurada

In [ ]:
# Configuración de SerpAPI para Google Shopping
SERPAPI_KEY = "API"  # Reemplazamos con nuestra API key

params = {
    "engine": "google_shopping",
    "format": "json",
    "hl": "es",           # Idioma: español
    "gl": "us",           # Geolocalización: US (mejor catálogo de suplementos)
    "api_key": SERPAPI_KEY
}

endpoint = "https://serpapi.com/search"

#Función de Extracción de Datos
Esta función se encarga de hacer las consultas a Google Shopping a través de SerpAPI.
### Funcionalidades:
-  Manejo robusto de errores de conexión
-  Timeout de 30 segundos para evitar bloqueos
-  Parámetros configurables para paginación
-  Validación de respuestas HTTP


In [ ]:
def get_supplements_info(keyword, start=0, num_results=20):
    """
    Obtiene información de suplementos deportivos desde Google Shopping

    Args:
        keyword (str): Término de búsqueda (ej: "proteína whey")
        start (int): Posición inicial para paginación
        num_results (int): Número máximo de resultados a obtener

    Returns:
        dict: Respuesta JSON de SerpAPI con productos encontrados
        None: Si hay errores de conexión o API
    """
    search_params = params.copy()
    search_params["q"] = keyword
    search_params["start"] = start
    search_params["num"] = num_results

    try:
        response = requests.get(endpoint, params=search_params, timeout=30)

        if response.status_code == 200:
            return response.json()
        else:
            print(f"❌ Error en la búsqueda: {response.status_code}")
            print(f"   Respuesta: {response.text[:200]}...")
            return None

    except requests.exceptions.Timeout:
        print(f"⏰ Timeout en la búsqueda de: {keyword}")
        return None
    except requests.exceptions.ConnectionError:
        print(f"🌐 Error de conexión para: {keyword}")
        return None
    except Exception as e:
        print(f"❌ Error inesperado: {e}")
        return None

#Configuración del Productor Kafka

 Establecemos la conexión con el broker de Kafka local.

 ### Configuración:
-  **bootstrap_servers**: Dirección del broker Kafka
-  **value_serializer**: Convierte objetos Python a JSON UTF-8
-  **Manejo de errores**: Validación de conexión antes de proceder

In [ ]:
# Configurar y validar conexión con Kafka
try:
    producer = KafkaProducer(
        bootstrap_servers=["localhost:9092"],
        value_serializer=lambda m: json.dumps(m, ensure_ascii=False).encode("utf-8"),
        retries=3,                    # Reintentos automáticos
        acks='all',                   # Confirmación de todos los replicas
        compression_type='gzip'       # Compresión para mejor performance
    )
    print("✅ Conexión establecida con Kafka")
    print(f"📡 Broker: localhost:9092")

except Exception as e:
    print("❌ Error: Kafka broker no disponible")
    print("🔧 Solución: Asegúrate que Kafka está ejecutándose en localhost:9092")
    print(f"   Comando: bin/kafka-server-start.sh config/server.properties")
    raise e

# Configuración del tópico
topic = "supplement_products"
print(f"📊 Tópico configurado: {topic}")

# Categorías de Suplementos a Procesar

 Definimos las categorías más importantes del mercado de suplementos deportivos.

 ### Categorías incluidas:
1.  **Proteínas**: Whey y vegana para diferentes públicos
2.  **Performance**: Creatina, BCAAs, pre-entrenos para rendimiento
3.  **Recuperación**: Glutamina para recuperación muscular
4.  **Salud General**: Multivitamínicos, Omega-3
5.  **Pérdida de Peso**: Quemadores de grasa
6.  **Conveniencia**: Barras proteicas para consumo fácil

In [ ]:
supplement_categories = [
    "proteína whey",           # Proteína más popular del mercado
    "creatina monohidrato",    # Suplemento más estudiado científicamente
    "pre-entrenamiento",       # Energizantes y potenciadores de rendimiento
    "bcaa",                    # Aminoácidos de cadena ramificada
    "glutamina",              # Recuperación y sistema inmune
    "quemadores de grasa",     # Termogénicos y fat burners
    "multivitamínicos",        # Suplementos de micronutrientes
    "omega 3",                # Ácidos grasos esenciales
    "proteína vegana",         # Alternativa plant-based
    "barras proteicas"         # Suplementos convenientes
]

print(f"🎯 Categorías a procesar: {len(supplement_categories)}")
for i, category in enumerate(supplement_categories, 1):
    print(f"   {i:2d}. {category}")

# Procesamiento Principal y Envío a Kafka

### Esta sección ejecuta el loop principal que:
1. Itera sobre cada categoría de suplementos
2. Extrae productos de Google Shopping via SerpAPI
3. Procesa y estructura los datos
4. Envía cada producto como mensaje a Kafka

### Estructura del mensaje:
- **type**: Identificador del tipo de mensaje
- **category**: Categoría del suplemento
- **timestamp**: Marca temporal de procesamiento
- **data**: Información completa del producto

In [ ]:
# Estadísticas del procesamiento
total_products_sent = 0
successful_categories = 0
failed_categories = []

print("\n" + "="*60)
print("🚀 INICIANDO PROCESAMIENTO DE CATEGORÍAS")
print("="*60)

for category_index, category in enumerate(supplement_categories, 1):
    print(f"\n🔍 [{category_index}/{len(supplement_categories)}] Procesando: {category}")
    print("-" * 50)

    # Obtener datos de SerpAPI
    data = get_supplements_info(category, num_results=40)  # Aumentamos a 40 para mejor cobertura

    if data and "shopping_results" in data:
        products = data["shopping_results"]
        products_count = len(products)
        print(f"   🛍️ Encontrados: {products_count} productos")

        # Contador para esta categoría
        category_sent = 0

        # Procesar y enviar cada producto
        for product_index, product in enumerate(products, 1):
            try:
                # Estructurar el mensaje
                message = {
                    "type": "supplement_product",
                    "search_category": category,
                    "timestamp": datetime.now().isoformat(),
                    "metadata": {
                        "api_source": "serpapi_google_shopping",
                        "processed_at": datetime.now().isoformat(),
                        "category_index": category_index,
                        "product_index": product_index
                    },
                    # Datos del producto
                    "position": product.get("position"),
                    "title": product.get("title"),
                    "price": product.get("price"),
                    "extracted_price": product.get("extracted_price"),
                    "source": product.get("source"),
                    "rating": product.get("rating"),
                    "reviews": product.get("reviews"),
                    "link": product.get("product_link"),
                    "image": product.get("thumbnail"),
                    "brand": product.get("brand", "N/A"),
                    "delivery": product.get("delivery", "N/A")
                }

                # Enviar a Kafka
                producer.send(topic, value=message)

                # Mostrar progreso
                title_truncated = message.get('title', 'Sin título')[:50]
                price = message.get('price', 'N/A')
                source = message.get('source', 'N/A')

                print(f"   📦 [{product_index:2d}/{products_count}] {title_truncated}...")
                print(f"       💰 {price} | 🏪 {source}")

                category_sent += 1
                total_products_sent += 1

                # Pausa para evitar saturación
                sleep(0.3)

            except Exception as e:
                print(f"   ❌ Error procesando producto {product_index}: {e}")
                continue

        successful_categories += 1
        print(f"   ✅ Categoría completada: {category_sent} productos enviados")

    elif not data:
        print(f"   ❌ No se pudieron obtener datos para: {category}")
        failed_categories.append(category)
    else:
        print(f"   ⚠️ No se encontraron productos para: {category}")
        failed_categories.append(category)

    # Pausa entre categorías para respetar rate limits
    if category_index < len(supplement_categories):
        print(f"   ⏳ Pausa de 2 segundos antes de la siguiente categoría...")
        sleep(2)

# Finalización y Estadísticas

### Proceso final que:
- Asegura que todos los mensajes se envíen (flush)
- Muestra estadísticas detalladas del procesamiento
- Reporta categorías exitosas y fallidas
- Proporciona métricas para monitoreo

In [ ]:
# Asegurar que todos los mensajes se envíen
producer.flush()
print("\n" + "="*60)
print("🎯 PROCESAMIENTO COMPLETADO")
print("="*60)

print(f"📊 Estadísticas finales:")
print(f"   ✅ Categorías procesadas exitosamente: {successful_categories}/{len(supplement_categories)}")
print(f"   📦 Total productos enviados: {total_products_sent}")
print(f"   📊 Promedio productos por categoría: {total_products_sent/len(supplement_categories):.1f}")
print(f"   🎯 Tópico utilizado: {topic}")

if failed_categories:
    print(f"\n⚠️ Categorías con problemas:")
    for failed in failed_categories:
        print(f"   - {failed}")

print(f"\n🕐 Procesamiento iniciado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"💾 Los datos están listos para ser consumidos desde Kafka")

# Cerrar el productor
producer.close()
print("🔌 Conexión con Kafka cerrada correctamente")